In [1]:
import pandas as pd
import numpy as np
import nltk
import regex
import re
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
import stanza
stanza.download('en') # download English model
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

2021-03-13 16:08:45 INFO: Downloading default packages for language: en (English)...
2021-03-13 16:08:46 INFO: File exists: C:\Users\vibkr\stanza_resources\en\default.zip.
2021-03-13 16:08:49 INFO: Finished downloading models and saved to C:\Users\vibkr\stanza_resources.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vibkr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vibkr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vibkr\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def feature_extraction(txt, nlp):

    sentList = nltk.sent_tokenize(txt)

    retlist = [];
    
    for line in sentList:
        
        txt_list = nltk.word_tokenize(line)
        taggedList = nltk.pos_tag(txt_list)
        
        newwordList = []
        flag = 0
        for i in range(0,len(taggedList)-1):
            if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"):
                newwordList.append(taggedList[i][0]+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                    flag=0
                    continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    newwordList.append(taggedList[i+1][0])
        finaltxt = ' '.join(word for word in newwordList)
    
    
        stop_words = set(stopwords.words('english'))
        new_txt_list = nltk.word_tokenize(finaltxt)
        wordsList = [w for w in new_txt_list if not w in stop_words]
        taggedList = nltk.pos_tag(wordsList)
        
        doc = nlp(finaltxt)
        dep_node = []
        try:
            for dep_edge in doc.sentences[0].dependencies:
                dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])
            for i in range(0, len(dep_node)):
                if (int(dep_node[i][1]) != 0):
                    dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]
        except:
            pass;
        
        #print(dep_node)
        
        featureList = []
        categories = []
        for i in taggedList:
            if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
                featureList.append(list(i))
                categories.append(i[0])
        #print(featureList)
        #print(categories)
        
        
        fcluster = []
        for i in featureList:
            filist = []
            for j in dep_node:
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in [
                    # Different types of words that are identified as potential features
                    "nsubj",
                    "acl:relcl",
                    "obj",
                    "dobj",
                    "agent",
                    "advmod",
                    "amod",
                    "neg",
                    "prep_of",
                    "acomp",
                    "xcomp",
                    "compound"
                ])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])
        print(fcluster) 
        
        # Remove all features with no sentiment word:
        
        retlist.append(fcluster)
    return retlist;
    

In [3]:
def do_extraction(df, nlp, feat_count, feat_sent, content_str = "Content"):
    idx = 0;
    # Replace "" with nan's for removal
    df[content_str].replace('', np.nan, inplace=True)
    df.dropna(subset=[content_str], inplace=True)
    review_list = df[content_str].to_list()
    #feat_count = dict()
    #feat_sent = dict()
    #nlp = stanza.Pipeline('en')
    
    
    
    
    print(" Processing : " , df.shape[0], "rows of data")
    for review in tqdm(review_list):
        print("Review Number : ", idx);
        
        # Some data pre-processing
        
        review = review.lower()
        
        # Merge hyphenated words
        separate = review.split('-')
        review = ''.join(separate)
        
        # Remove non-alphabets
        review = re.sub(r'[^a-z\s\t.!?]', '', review)
        
        idx += 1;
        if idx >= df.shape[0]:
            break;
        try:
            output = feature_extraction(review, nlp);
        except:
            pass;
        for sent in output:
            for pair in sent:
                print(pair)
                if pair[0] in feat_sent:
                    if pair[1] is not None:
                        flist = feat_sent[pair[0]]
                        if isinstance(pair[1], list):
                            for i in pair[1]:
                                flist.append(i)
                        else:
                            flist.append(pair[1])
                        feat_sent[pair[0]] = flist;
                else:
                    if pair[1] is not None:
                        flist = pair[1]
                    else:
                        flist = list()
                    feat_sent[pair[0]] = flist;
                
                if pair[0] in feat_count:
                    feat_count[pair[0]] = feat_count[pair[0]] + 1;
                else:
                    feat_count[pair[0]] = 1
    
    return feat_count, feat_sent;

In [4]:
def get_sentiment(feat_count, feat_sent, nlp):

    sentiment_score = dict()

    # Delete features with no descriptors
    cob = feat_sent.copy()
    for feat in cob.keys():
        #print(cob[feat])
        
        if cob[feat] == []:
            del feat_sent[feat]
        else:
            feat_sent[feat] = ' ,'.join(feat_sent[feat])

    # Run pre-built sentiment score and take avg of all descriptors
    for f in tqdm(feat_sent.keys()):
        print("Calculating Sentiment for: ", f);
        ssum = 0;
        for g in feat_sent[f]:
            try:
                doc = nlp(g);

                for i in doc.sentences:

                        #print(i.sentiment)
                        ssum += i.sentiment;
            except:
                pass;

        sentiment_score[f] = ssum / len(b[f])

    adf = pd.DataFrame.from_dict(feat_count, orient='index', columns=['Freq'])
    adf.sort_values(by="Freq", ascending=False, inplace = True)

    

    avg_sent = pd.DataFrame.from_dict(sentiment_score, orient='index', columns=["Avg_sent"])
    desc_words = pd.DataFrame.from_dict(feat_sent, orient="index", columns=["Descriptors"])
    
    avg_sent = avg_sent.merge(desc_words, left_index=True, right_index=True)
    
    
    final_sent = avg_sent.merge(adf, left_index=True, right_index=True)
    final_sent.sort_values(by="Freq", ascending=False, inplace=True)
    return final_sent;

In [5]:
rdr = pd.read_csv('../ScrapedOutput/cmpb.csv')

nlp = stanza.Pipeline('en')
a = dict()
b = dict()
a, b = do_extraction(rdr, nlp, a, b)

2021-03-13 16:08:49 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-03-13 16:08:49 INFO: Use device: cpu
2021-03-13 16:08:49 INFO: Loading: tokenize
2021-03-13 16:08:49 INFO: Loading: pos
2021-03-13 16:08:50 INFO: Loading: lemma
2021-03-13 16:08:50 INFO: Loading: depparse
2021-03-13 16:08:50 INFO: Loading: sentiment
2021-03-13 16:08:50 INFO: Loading: ner
2021-03-13 16:08:51 INFO: Done loading processors!
  0%|                                                                                           | 0/52 [00:00<?, ?it/s]

 Processing :  52 rows of data
Review Number :  0
[['professional', ['very', 'people']], ['people', ['professional', 'there']]]


  2%|█▌                                                                                 | 1/52 [00:00<00:25,  1.99it/s]

[['patient', ['very', 'recommended']], ['kind', ['respectful']], ['respectful', ['kind']], ['..', []], ['smooth', ['very']], ['medical', ['checkup']], ['checkup', ['medical']]]
['professional', ['very', 'people']]
['people', ['professional', 'there']]
['patient', ['very', 'recommended']]
['kind', ['respectful']]
['respectful', ['kind']]
['..', []]
['smooth', ['very']]
['medical', ['checkup']]
['checkup', ['medical']]
Review Number :  1


  4%|███▏                                                                               | 2/52 [00:00<00:23,  2.13it/s]

...

[['place', []], ['well', ['kept']], ['people', ['kept']], ['unbelievably', ['rude']], ['rude', ['unbelievably']]]
['place', []]
['well', ['kept']]
['people', ['kept']]
['unbelievably', ['rude']]
['rude', ['unbelievably']]
Review Number :  46
[['bad', ['service']], ['service', ['bad']]]
['bad', ['service']]
['service', ['bad']]
Review Number :  47


 92%|███████████████████████████████████████████████████████████████████████████▋      | 48/52 [00:15<00:00,  6.12it/s]

[['interestingly', ['enough']], ['enough', ['interestingly', 'removed']], ['negative', ['reviews']], ['reviews', ['negative']]]
['interestingly', ['enough']]
['enough', ['interestingly', 'removed']]
['negative', ['reviews']]
['reviews', ['negative']]
Review Number :  48
[['bad', ['service']], ['service', ['bad']]]
['bad', ['service']]
['service', ['bad']]
Review Number :  49
[['sheat', []]]


 98%|████████████████████████████████████████████████████████████████████████████████▍ | 51/52 [00:16<00:00,  3.11it/s]

[['dirty', ['pigs']], ['pigs', ['dirty', 'training']], ['step', []], ['minefields', []]]
['sheat', []]
['dirty', ['pigs']]
['pigs', ['dirty', 'training']]
['step', []]
['minefields', []]
Review Number :  50
[['gncpresent', []]]
['gncpresent', []]
Review Number :  51


In [ ]:
fin = get_sentiment(a, b, nlp)

  0%|                                                                                          | 0/197 [00:00<?, ?it/s]

Calculating Sentiment for:  professional


  1%|▍                                                                                 | 1/197 [00:01<03:32,  1.08s/it]

Calculating Sentiment for:  people


  1%|▊                                                                                 | 2/197 [00:05<09:16,  2.85s/it]

...

Calculating Sentiment for:  staff


 25%|████████████████████▏                                                            | 49/197 [00:47<04:26,  1.80s/it]

Calculating Sentiment for:  bad


 25%|████████████████████▌                                                            | 50/197 [00:52<07:08,  2.91s/it]

Calculating Sentiment for:  personal


In [ ]:
fin
fin.to_csv("onenineseven.csv")

In [ ]:
def get_tfidf_features(df, content_str = "Content", min_ = 2, max_ = 0.5, ngramrange = (1,2)):
    
    # Replace "" with nan's for removal
    #df[content_str].replace('', np.nan, inplace=True)
    #df.dropna(subset=[content_str], inplace=True)
    #stop_words = set(stopwords.words('english'))
    #df[content_str] = df[content_str].apply(lambda x: ''.join([word for word in x.split() if word not in (stop_words)]))
    
    review_list = df[content_str].to_list()
    #feat_count = dict()
    #feat_sent = dict()
    #nlp = stanza.Pipeline('en')

        
    #print(review_list)
    tfidf = TfidfVectorizer(min_df = min_, max_df = max_, ngram_range = ngramrange);
    features = tfidf.fit_transform(review_list);
    q = pd.DataFrame(features.todense(), columns=tfidf.get_feature_names())
    
    return list(q.columns)

In [ ]:
def refine_features(originaldf, sentimentdf):
    tfidf_output = get_tfidf_features(originaldf)
    sentimentdf = sentimentdf.reset_index()
    ft_extract = set(sentimentdf['index']);
    tfidf_extract = set(tfidf_output)
    
    intersecting_features = ft_extract.intersection(tfidf_extract)
    
    return_df = sentimentdf
    return_df = return_df.loc[return_df['index'].isin(list(intersecting_features))]
    print("Number of extracted features:")
    print("Initial = ", len(ft_extract), " TFIDF = ", len(intersecting_features), " Final after intersection = ", return_df.shape[0])
    return return_df
    
    

In [ ]:
rt = refine_features(rdr, fin)
rt

In [ ]:
#rt.to_csv("finalFeatures.csv")